In [148]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend

In [149]:
df = pd.read_csv('../data/df_for_model.csv')
# df['start_time'] = df['start_time']//1000
df = df[df['result'] != 0.5]
numeric_predictors = ['diff', 'color',
                      'game_time', 'start_time', 'weekday', 'day']
X = df[numeric_predictors]
y = np.array(df['result'])

In [150]:
X.shape

(2024, 6)

In [151]:
y.shape

(2024,)

In [152]:
X_train = X[:1820]
y_train = y[:1820]
X_test = X[1820:]
y_test = y[1820:]

In [153]:
X_train.shape

(1820, 6)

In [154]:
y_train.shape

(1820,)

In [155]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [156]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [157]:
def _classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform',
                         activation='relu', input_dim=6))
    classifier.add(
        Dense(units=72, kernel_initializer='uniform', activation='softsign'))
    classifier.add(
        Dense(units=6, kernel_initializer='uniform', activation='relu'))
    
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [158]:
classifier = _classifier()

In [159]:
classifier.fit(X_train, y_train, batch_size=10, epochs=150)

Epoch 1/150
1820/1820 [==============================] - 1s 654us/step - loss: 0.6267 - acc: 0.6297
Epoch 2/150
1820/1820 [==============================] - 0s 270us/step - loss: 0.5678 - acc: 0.7055
Epoch 3/150
1820/1820 [==============================] - 0s 253us/step - loss: 0.5644 - acc: 0.6978
Epoch 4/150
1820/1820 [==============================] - 0s 253us/step - loss: 0.5631 - acc: 0.7027
Epoch 5/150
1820/1820 [==============================] - 0s 246us/step - loss: 0.5610 - acc: 0.7038
Epoch 6/150
1820/1820 [==============================] - 0s 259us/step - loss: 0.5609 - acc: 0.7082
Epoch 7/150
1820/1820 [==============================] - 0s 260us/step - loss: 0.5600 - acc: 0.7082
Epoch 8/150
1820/1820 [==============================] - 0s 256us/step - loss: 0.5580 - acc: 0.7099
Epoch 9/150
1820/1820 [==============================] - 0s 263us/step - loss: 0.5579 - acc: 0.7099
Epoch 10/150
1820/1820 [==============================] - 0s 272us/step - loss: 0.5574 - acc: 0.7077

1820/1820 [==============================] - 1s 289us/step - loss: 0.5452 - acc: 0.7132
Epoch 83/150
1820/1820 [==============================] - 0s 264us/step - loss: 0.5436 - acc: 0.7170
Epoch 84/150
1820/1820 [==============================] - 1s 280us/step - loss: 0.5462 - acc: 0.7154
Epoch 85/150
1820/1820 [==============================] - 0s 271us/step - loss: 0.5440 - acc: 0.7104
Epoch 86/150
1820/1820 [==============================] - 1s 279us/step - loss: 0.5441 - acc: 0.7077
Epoch 87/150
1820/1820 [==============================] - 0s 261us/step - loss: 0.5447 - acc: 0.7104
Epoch 88/150
1820/1820 [==============================] - 0s 272us/step - loss: 0.5454 - acc: 0.7143
Epoch 89/150
1820/1820 [==============================] - 0s 257us/step - loss: 0.5439 - acc: 0.7132
Epoch 90/150
1820/1820 [==============================] - 1s 279us/step - loss: 0.5439 - acc: 0.7093
Epoch 91/150
1820/1820 [==============================] - 0s 239us/step - loss: 0.5440 - acc: 0.7192
Epo

In [160]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [161]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[70, 35],
       [33, 66]])

In [162]:
(70+66)/(82+23+32+67)*100

66.66666666666666

In [128]:
classifier = KerasClassifier(
    build_fn=_classifier, batch_size=13, epochs=175)

In [129]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-4)

Epoch 1/175


KeyboardInterrupt: 

In [ ]:
accus

In [ ]:
accu_mean = accus.mean()
accu_mean

In [ ]:
accu_var = accus.std()
accu_var

In [121]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform',
                         activation='relu', input_dim=6))
    classifier.add(Dropout(rate=0.22))
    classifier.add(
        Dense(units=64, kernel_initializer='uniform', activation='softsign'))
    classifier.add(Dropout(rate=0.15))
    classifier.add(
        Dense(units=64, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.06))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [122]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [123]:
parameters = {'batch_size': [7, 13, 21], 'nb_epoch': [100, 140, 175],
              'optimizer': ['rmsprop', 'sgd', 'Nadam', 'Adadelta', 'adamax', 'adam']}

In [124]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='accuracy', cv=10, n_jobs=-4)

In [125]:
grid_search = grid_search.fit(X=X_train, y=y_train)

Epoch 1/1
1638/1638 [==============================] - 2s 987us/step - loss: 0.6756 - acc: 0.5629
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6709 - acc: 0.6178
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6666 - acc: 0.6099
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6655 - acc: 0.5922
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6664 - acc: 0.6074
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6790 - acc: 0.5788
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6683 - acc: 0.6136
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6778 - acc: 0.5757
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6781 - acc: 0.5702
Epoch 1/1
1638/1638 [==============================] - 2s 1ms/step - loss: 0.6625 - acc: 0.6197
Epoch 1/1
1638/1638 [=================

1638/1638 [==============================] - 10s 6ms/step - loss: 0.6494 - acc: 0.6221
Epoch 1/1
1638/1638 [==============================] - 9s 6ms/step - loss: 0.6680 - acc: 0.5812
Epoch 1/1
1638/1638 [==============================] - 10s 6ms/step - loss: 0.6512 - acc: 0.6093
Epoch 1/1
1638/1638 [==============================] - 10s 6ms/step - loss: 0.6574 - acc: 0.5879
Epoch 1/1
1638/1638 [==============================] - 10s 6ms/step - loss: 0.6552 - acc: 0.6062
Epoch 1/1
1638/1638 [==============================] - 10s 6ms/step - loss: 0.6404 - acc: 0.6245
Epoch 1/1
1638/1638 [==============================] - 10s 6ms/step - loss: 0.6626 - acc: 0.5824
Epoch 1/1
1638/1638 [==============================] - 11s 7ms/step - loss: 0.6521 - acc: 0.6343
Epoch 1/1
1638/1638 [==============================] - 12s 7ms/step - loss: 0.6588 - acc: 0.5775
Epoch 1/1
1638/1638 [==============================] - 13s 8ms/step - loss: 0.6577 - acc: 0.6111
Epoch 1/1
1638/1638 [====================

1638/1638 [==============================] - 17s 10ms/step - loss: 0.6932 - acc: 0.4976
Epoch 1/1
1638/1638 [==============================] - 17s 10ms/step - loss: 0.6932 - acc: 0.5073
Epoch 1/1
1638/1638 [==============================] - 17s 10ms/step - loss: 0.6909 - acc: 0.5659
Epoch 1/1
1638/1638 [==============================] - 17s 10ms/step - loss: 0.6923 - acc: 0.5287
Epoch 1/1
1638/1638 [==============================] - 17s 11ms/step - loss: 0.6906 - acc: 0.5226
Epoch 1/1
1638/1638 [==============================] - 17s 11ms/step - loss: 0.6911 - acc: 0.5360
Epoch 1/1
1638/1638 [==============================] - 17s 11ms/step - loss: 0.6915 - acc: 0.5195
Epoch 1/1
1638/1638 [==============================] - 17s 11ms/step - loss: 0.6926 - acc: 0.4976
Epoch 1/1
1638/1638 [==============================] - 17s 11ms/step - loss: 0.6921 - acc: 0.5696
Epoch 1/1
1638/1638 [==============================] - 17s 11ms/step - loss: 0.6893 - acc: 0.5525
Epoch 1/1
1638/1638 [=========

1638/1638 [==============================] - 27s 17ms/step - loss: 0.6469 - acc: 0.6136
Epoch 1/1
1638/1638 [==============================] - 27s 17ms/step - loss: 0.6566 - acc: 0.6020
Epoch 1/1
1638/1638 [==============================] - 28s 17ms/step - loss: 0.6494 - acc: 0.6300
Epoch 1/1
1638/1638 [==============================] - 28s 17ms/step - loss: 0.6492 - acc: 0.6105
Epoch 1/1
1638/1638 [==============================] - 27s 17ms/step - loss: 0.6546 - acc: 0.5965
Epoch 1/1
1638/1638 [==============================] - 28s 17ms/step - loss: 0.6562 - acc: 0.6007
Epoch 1/1
1638/1638 [==============================] - 27s 17ms/step - loss: 0.6932 - acc: 0.4780
Epoch 1/1
1638/1638 [==============================] - 28s 17ms/step - loss: 0.6932 - acc: 0.5006
Epoch 1/1
1638/1638 [==============================] - 27s 17ms/step - loss: 0.6932 - acc: 0.4890
Epoch 1/1
1638/1638 [==============================] - 28s 17ms/step - loss: 0.6932 - acc: 0.4811
Epoch 1/1
1638/1638 [=========

In [126]:
best_params = grid_search.best_params_
best_params

{'batch_size': 13, 'nb_epoch': 175, 'optimizer': 'Nadam'}

In [127]:
best_accu = grid_search.best_score_
best_accu

0.7104395604395605